In [1]:
from bookimed_lib import *

In [2]:
FEATURES = [
    "top_oper_cli", 
    "gen_oper_cli",
    "top_proc_cli", 
    "gen_proc_cli", 
]

In [3]:
def extract_data(clinic, clinic_estim):
    all_proc = clinic['procedures']['diagnostics'] + clinic['procedures']['operations']
    X = []
    X += [[
        #sum([int(i['top']) for i in clinic['procedures']['operations']]), #top_oper_cli
        sum([int(i['top']) for i in clinic['procedures']['operations']]) / float(len(clinic['procedures']['operations'])),
        len(clinic['procedures']['operations']), #gen_oper_cli
        #sum([int(i['top']) for i in all_proc]), #top_proc_cli
        sum([int(i['top']) for i in all_proc]) / float(len(all_proc)),
        len(all_proc), #gen_proc_cli
        ]]
    y=[int(clinic_estim['variety'])]
    return [X, y]

In [4]:
def normale(mat):
    for i in [0,1,2,3]:
        mass = [k[i] for k in mat]
        m = max(mass)
        if m>0:
            for j in mat:
                j[i] = round(j[i] / float(m), 4)

In [5]:
#2
def normale(mat):
    for i in range(0,len(mat[0])):
        mass = [k[i] for k in mat]
        mean = np.mean(mass)
        mx = max(mass)
        mn = min(mass)
        if mx!=mn:
            for j in mat:
                #print j[i], j[i] - mn, mx-mn
                j[i] = round((j[i] - mn) / float(mx-mn), 4)
                #print j[i]

In [6]:
def process_with(X,y, info=False, short=False, return_short = False, new_coef = [], ts=0.2):
    train_X, test_X, train_y, test_y = cross_validation.train_test_split(X, y, test_size = ts, random_state = 3)
    regr = linear_model.LinearRegression(fit_intercept=False, normalize=False)
    if new_coef != []:
        print "new coefs"
        regr.coef_ = new_coef
    regr.fit(train_X, train_y)
    if info:
        print "Total: %d, train: %d, test: %d" %(len(X), len(train_X), len(test_X))
        print("Residual sum of squares: %.2f"% np.mean((regr.predict(test_X) - test_y) ** 2))
        print("Train absolute: %.2f"% np.mean(abs(regr.predict(train_X) - train_y)))
        print("Test absolute: %.2f"% np.mean(abs(regr.predict(test_X) - test_y)))
        print("Absolute to mean: %.2f%%"% (np.mean(abs(regr.predict(test_X) - test_y))/np.mean(test_y)*100))
        print('Train variance score: %.2f' % regr.score(train_X, train_y))
        print('Test variance score: %.2f' % regr.score(test_X, test_y))
    if short:
        print "Total: %d, train: %d, test: %d" %(len(X), len(train_X), len(test_X))
        print "%.3f" % np.mean(abs(regr.predict(train_X) - train_y))
        print "%.3f" % np.mean(abs(regr.predict(test_X) - test_y))
        print "%.3f" % (np.mean(abs(regr.predict(test_X) - test_y))/np.mean(test_y)*100)
        print "%.3f" % regr.score(train_X, train_y)
        print "%.3f" % regr.score(test_X, test_y)
    if return_short:
        return np.mean(abs(regr.predict(test_X) - test_y)),regr.score(test_X, test_y)
    for i in regr.coef_:
        print "%.3f" % i
    #print "%.3f" % regr.intercept_
    return regr

In [7]:
X_all, X_1, X_2, X_3, X_4, X_5, y_all, y_1, y_2, y_3, y_4, y_5 = get_X_sets(extract_data)
normale(X_all)
normale(X_1)
normale(X_2)
normale(X_3)
normale(X_4)
normale(X_5)

In [8]:
for i in range(1,6):
    print i
    regr = process_with(X_all, y_all, info=True, ts=i/10.0)
    print

1
Total: 49, train: 44, test: 5
Residual sum of squares: 4.97
Train absolute: 1.95
Test absolute: 1.38
Absolute to mean: 16.85%
Train variance score: -1.01
Test variance score: -30.04
1.549
-9.067
6.772
16.727

2
Total: 49, train: 39, test: 10
Residual sum of squares: 4.17
Train absolute: 2.06
Test absolute: 1.54
Absolute to mean: 18.50%
Train variance score: -1.03
Test variance score: -5.84
2.381
-9.953
6.094
17.105

3
Total: 49, train: 34, test: 15
Residual sum of squares: 5.66
Train absolute: 1.98
Test absolute: 1.91
Absolute to mean: 23.70%
Train variance score: -0.74
Test variance score: -8.50
2.680
-8.556
5.817
16.125

4
Total: 49, train: 29, test: 20
Residual sum of squares: 5.36
Train absolute: 2.09
Test absolute: 1.88
Absolute to mean: 22.92%
Train variance score: -0.65
Test variance score: -7.13
2.320
-11.045
6.134
18.751

5
Total: 49, train: 24, test: 25
Residual sum of squares: 4.65
Train absolute: 2.32
Test absolute: 1.73
Absolute to mean: 20.95%
Train variance score: -0.7

In [12]:
regr = process_with(X_all, y_all, info=True, ts=0.2)

Total: 49, train: 39, test: 10
Residual sum of squares: 4.17
Train absolute: 2.06
Test absolute: 1.54
Absolute to mean: 18.50%
Train variance score: -1.03
Test variance score: -5.84
2.381
-9.953
6.094
17.105


In [10]:
regr.coef_

array([  2.38142122,  -9.95300862,   6.09372163,  17.10517242])

In [ ]:
print "Features sorted by their score:"
#f = sorted(zip(map(lambda x: round(x, 4), regr.coef_), FEATURES), reverse=True)
f = zip(map(lambda x: round(x, 4), regr.coef_), FEATURES)
print regr.intercept_
print len(FEATURES)
for i in f:
    print "%4f \t %s" %(i[0], i[1])
    #print i[0]

In [ ]:
f = sorted(zip(map(lambda x: round(x, 4), regr.coef_), FEATURES), reverse=True)
ind = [map(lambda x: x[1], f).index(i) for i in FEATURES]
for i in ind:
    print i+1

In [ ]:
import warnings
warnings.filterwarnings("ignore")
for i in range(0,len(y_4)):
    print "Real: %f \t Predicted: %f" %(y_4[i], regr.predict(X_4[i]))
    print X_4[i]
    print sum(np.array(X_4[i]) * np.array([-3.094, -7.545, 11.025, 9.011]))
    print

In [ ]:
plt.plot([i[2] for i in X_all])
plt.show()

In [ ]:
print FEATURES[8]
[i[22] for i in X]

In [ ]:
for k in range(0,26):
    print FEATURES[k], 10 - [i[k] for i in X].count(0)